in this file a potenitla api is developed for the tramdag 

In [ ]:
"config.json" has to be created either manually or by running createw_config.ipynb first!

In [ ]:
cfg=TramDagConfig.load("config.json") # basically just loads a json file to a dictionary


# checks all specifications and throws warnings
# returns a dict

In [ ]:
train_data=TramDagDataset.from_dataframe(df_train,which_set='train',cfg) 

# additonal arguments could be batch_size, num_workers, pin_memory etc 

# returs a dictionary with {'x1':dataloader1,'x2':dataloader2,...}

In [ ]:
td_model = TramDagModel.from_config(cfg) 

# builds the models according to the config
# returns a dict of TramModels for each node {'x1':model1,'x2':model2,...}

In [ ]:
td_trainer = TramDagTrainer(cfg, learning_rate,  device="auto")

# returns a trainer object which can train all models in the dag independently



td_fit=td_trainer.fit(td_model, train_data, val_data, train_list=['x1','x2','x3'], epochs=100, callbacks=[])


